# Word2vec 
Natural-language processing (NLP) is an area of computer science and artificial intelligence concerned with the interactions between computers and human (natural) languages, in particular how to program computers to fruitfully process large amounts of natural language data. <br/><br/>

The idea behind Word2Vec is pretty simple. We’re making an assumption that the meaning of a word can be inferred by the company it keeps. This is analogous to the saying, “show me your friends, and I’ll tell who you are”. If you have two words that have very similar neighbors (meaning: the context in which its used is about the same), then these words are probably quite similar in meaning or are at least related. For example, the words shocked, appalled and astonished are usually used in a similar context.<br/><br/>

<img src='./resources/NN model.png' alt='boosting' style="width: 500px;"/>
The `projection layer` maps the discrete word indices of an n-gram context to a continuous vector space.The projection layer is shared such that for contexts containing the same word multiple times, the same set of weights is applied to form each part of the projection vector. This organization effectively increases the amount of data available for training the `projection layer weights` since each word of each context training pattern individually contributes changes to the weight values.<br/><br/>

The `hidden layer` processes the output of the projection layer and is also created with a number of neurons specified in the topology configuration file.<br/><br/>

<img src='./resources/n-gram model.jpg' alt='boosting' style="width: 500px;"/>

### Count-Based Vector Space Model
Count-based vector space models heavily rely on the word frequency and co-occurrence matrix with the assumption that words in the same contexts share similar or related semantic meanings. The models map count-based statistics like co-occurrences between neighboring words down to a small and dense word vectors. PCA, topic models, and neural probabilistic language models are all good examples of this category.

### Count-Based models
Different from the count-based approaches, context-based methods build predictive models that directly target at predicting a word given its neighbors. The dense word vectors are part of the model parameters. The best vector representation of each word is learned during the model training process. <br/><br/>

<img src='./resources/CBOW and skip-gram.png' alt='boosting' style="width: 500px;"/><br/><br/><br/>

### Context-Based: Skip-Gram Model
<img src='./resources/word2vec-skip-gram.png' alt='boosting' style="width: 500px;"/>
<p style="text-align: center;"> Fig. 1. The skip-gram model. Both the input vector $x$ and the output $y$ are one-hot encoded word representations. The hidden layer is the word embedding of size $N$.</p><br/>

Given the vocabulary size $V$, we are about to learn word embedding vectors of size $N$. The model learns to predict one context word (output) using one target word (input) at a time.<br/>

According to Fig. 1,<br/>

* Both input word $w_{i}$ and the output word $w_{j}$ are one-hot encoded into binary vectors $x$ and $y$ of size $V$.
* First, the multiplication of the binary vector $x$ and the word embedding matrix $W$ of size $V×N$ gives us the embedding vector of the input word $w_{i}$: the i-th row of the matrix $W$.
* This newly discovered embedding vector of dimension $N$ forms the hidden layer.
* The multiplication of the hidden layer and the word context matrix $W′$ of size $N×W$ produces the output one-hot encoded vector $y$.
* The output context matrix $W′$ encodes the meanings of words as context, different from the embedding matrix $W$. `NOTE`: Despite the name, $W′$ is independent of $W$, not a transpose or inverse or whatsoever.

<br/><br/>

### Context-Based: Skip-Gram Model
The Continuous Bag-of-Words (CBOW) is another similar model for learning word vectors. It predicts the target word (i.e. “swing”) from source context words (i.e., “sentence should the sword”).根据上下文的词语预测当前词语的出现概率

<img src='./resources/word2vec-cbow.png' alt='boosting' style="width: 500px;"/>
<p style="text-align: center;"> Fig. 2. The CBOW model. Word vectors of multiple context words are averaged to get a fixed-length vector as in the hidden layer. Other symbols have the same meanings as in Fig 1.</p><br/>

Because there are multiple contextual words, we average their corresponding word vectors, constructed by the multiplication of the input vector and the matrix $W$. Because the averaging stage smoothes over a lot of the distributional information, some people believe the CBOW model is better for small dataset.



### Hierarchical Softmax
Morin and Bengio ([2005](https://www.iro.umontreal.ca/~lisa/pointeurs/hierarchical-nnlm-aistats05.pdf)) proposed hierarchical softmax to make the sum calculation faster with the help of a `Huffman tree` structure. The hierarchical softmax encodes the language model’s output softmax layer into a tree hierarchy, where each leaf is one word and each internal node stands for relative probabilities of the children nodes.
<img src='./resources/word2vec-hierarchical-softmax.png' alt='boosting' style="width: 500px;"/>
<p style="text-align: center;"> Fig. 3. An illustration of the hierarchical softmax binary tree. The leaf nodes in white are words in the vocabulary. The gray inner nodes carry information on the probabilities of reaching its child nodes. One path starting from the root to the leaf $w_{i}.n(w_{i},j)$
denotes the j-th node on this path.</p>

Each word $w_{i}$ has a unique path from the root down to its corresponding leaf. The probability of picking this word is equivalent to the probability of taking this path from the root down through the tree branches. Since we know the embedding vector $v_{n}$ of the internal node $n$, the probability of getting the word can be computed by the product of taking left or right turn at every internal node stop.

According to Fig. 3, the probability of one node is (σ is the sigmoid function):

$$p(turn\ right  \rightarrow ... w_{I}|n) = \sigma (v_{n}^{'T} v_{w_{I}})$$
$$p(turn\ left  \rightarrow ... w_{I}|n) =1- p(turn\ right|n)= \sigma (-v_{n}^{'T} v_{w_{I}})$$

The final probability of getting a context word $w_{O}$ given an input word $w_{I}$ is:
$$ p(w_{o}|w_{I}) = \prod_{k=1}^{L(w_{o})}\sigma (I_{turn}(n(w_{o},k),n(w_{o},k+1))\cdot v_{n(w_{o},k)}^{'T}v_{w_{I}}) $$

where $L(w_{O})$ is the depth of the path leading to the word $w_{O}$ and $I_{turn}$ is a specially indicator function which returns 1 if $n(w_{o},k+1)$ is the left child of $n(w_{O},k)$ otherwise -1. 

The internal nodes’ embeddings are learned during the model training. The tree structure helps greatly reduce the complexity of the denominator estimation from $O(V)$ (vocabulary size) to $O(log V)$ (the depth of the tree) at the training time. However, at the prediction time, we still to compute the probability of every word and pick the best, as we don’t know which leaf to reach for in advance.

A good tree structure is crucial to the model performance. Several handy principles are: group words by frequency like what is implemented by Huffman tree for simple speedup; group similar words into same or close branches (i.e. use predefined word clusters, WordNet).
